# Teil 3: Statistische Analyse & Visualisierung

**Ziel:**
- Histogramme der Feature-Verteilungen (Tumor vs. Normal)
- Korrelations-Heatmap zwischen Features
- Bildvergleich mit Entropie-Werten

**Projektanforderungen:**
1. Gepaartes Histogramm für Entropie (Tumor vs. Normal)
2. Korrelations-Heatmap der 3 Features
3. Abbildung mit Tumor- und Normal-Beispiel + Entropie-Werten

---

## 1. Imports & Daten laden

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io
from pathlib import Path

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries importiert")

Libraries importiert


In [2]:
# Pfade
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')
RESULTS_DIR = Path('../results/figures')

# DataFrame mit Features laden
df = pd.read_csv(PROCESSED_DIR / 'features_with_labels.csv')

print(f"Dataset geladen: {len(df)} Bilder")
print(f"\nLabel-Verteilung:")
print(df['label'].value_counts())

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/features_with_labels.csv'

## 2. Projektanforderung 1: Gepaartes Histogramm für Entropie

Zeigt die Verteilung der Entropie-Werte getrennt nach Tumor und Normal.

In [ ]:
# Gepaartes Histogramm
fig, ax = plt.subplots(figsize=(10, 6))

# Daten aufteilen
tumor_entropy = df[df['label'] == 'Tumor']['entropy']
normal_entropy = df[df['label'] == 'Normal']['entropy']

# Histogramme plotten
ax.hist(tumor_entropy, bins=15, alpha=0.6, label='Tumor', color='red', edgecolor='black')
ax.hist(normal_entropy, bins=15, alpha=0.6, label='Normal', color='green', edgecolor='black')

# Mittelwerte als vertikale Linien
ax.axvline(tumor_entropy.mean(), color='darkred', linestyle='--', linewidth=2, 
           label=f'Tumor Mean: {tumor_entropy.mean():.3f}')
ax.axvline(normal_entropy.mean(), color='darkgreen', linestyle='--', linewidth=2,
           label=f'Normal Mean: {normal_entropy.mean():.3f}')

ax.set_xlabel('Entropie', fontsize=12, fontweight='bold')
ax.set_ylabel('Häufigkeit', fontsize=12, fontweight='bold')
ax.set_title('Entropie-Verteilung: Tumor vs. Normal', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / '10_entropy_histogram_paired.png', dpi=150, bbox_inches='tight')
plt.show()

print("PROJEKTANFORDERUNG 1 erfüllt")
print(f"Plot gespeichert: results/figures/10_entropy_histogram_paired.png")
print(f"\nStatistik:")
print(f"  Tumor - Mean: {tumor_entropy.mean():.4f}, Std: {tumor_entropy.std():.4f}")
print(f"  Normal - Mean: {normal_entropy.mean():.4f}, Std: {normal_entropy.std():.4f}")
print(f"  Differenz: {abs(tumor_entropy.mean() - normal_entropy.mean()):.4f}")

## 3. Projektanforderung 2: Korrelations-Heatmap

Zeigt die Korrelation zwischen den 3 Textur-Features.

In [ ]:
# Korrelationsmatrix berechnen
feature_cols = ['entropy', 'variance', 'median']
corr_matrix = df[feature_cols].corr()

print("Korrelationsmatrix:")
print(corr_matrix)
print("\nInterpretation:")
print("  1.0 = perfekte positive Korrelation")
print("  0.0 = keine Korrelation")
print(" -1.0 = perfekte negative Korrelation")

In [ ]:
# Heatmap visualisieren
fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.3f',
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=1,
    cbar_kws={'label': 'Korrelationskoeffizient'},
    ax=ax
)

ax.set_title('Korrelations-Heatmap der Textur-Features', fontsize=14, fontweight='bold', pad=20)
ax.set_xticklabels(['Entropie', 'Varianz', 'Median'], fontsize=11)
ax.set_yticklabels(['Entropie', 'Varianz', 'Median'], fontsize=11, rotation=0)

plt.tight_layout()
plt.savefig(RESULTS_DIR / '11_correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print("PROJEKTANFORDERUNG 2 erfüllt")
print(f"Plot gespeichert: results/figures/11_correlation_heatmap.png")

## 4. Projektanforderung 3: Bildvergleich mit Entropie-Werten

Zeigt 1 Tumor- und 1 Normal-Beispiel mit ihren Entropie-Werten.

In [ ]:
# Bestes Beispiel finden: höchste Entropie-Differenz
tumor_high_entropy = df[df['label'] == 'Tumor'].nlargest(1, 'entropy').iloc[0]
normal_low_entropy = df[df['label'] == 'Normal'].nsmallest(1, 'entropy').iloc[0]

print("Ausgewählte Beispiele:")
print(f"\nTumor (hohe Entropie):")
print(f"  Datei: {tumor_high_entropy['filename']}")
print(f"  Entropie: {tumor_high_entropy['entropy']:.4f}")
print(f"  Varianz: {tumor_high_entropy['variance']:.6f}")
print(f"  Median: {tumor_high_entropy['median']:.4f}")

print(f"\nNormal (niedrige Entropie):")
print(f"  Datei: {normal_low_entropy['filename']}")
print(f"  Entropie: {normal_low_entropy['entropy']:.4f}")
print(f"  Varianz: {normal_low_entropy['variance']:.6f}")
print(f"  Median: {normal_low_entropy['median']:.4f}")

In [ ]:
# Bilder laden
tumor_img = io.imread(DATA_DIR / tumor_high_entropy['filename'])
normal_img = io.imread(DATA_DIR / normal_low_entropy['filename'])

# Visualisierung
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Tumor
axes[0].imshow(tumor_img)
axes[0].set_title(
    f"TUMOR\n\nEntropie: {tumor_high_entropy['entropy']:.4f}\nVarianz: {tumor_high_entropy['variance']:.6f}\nMedian: {tumor_high_entropy['median']:.4f}",
    fontsize=12,
    fontweight='bold',
    color='red'
)
axes[0].axis('off')

# Normal
axes[1].imshow(normal_img)
axes[1].set_title(
    f"NORMAL\n\nEntropie: {normal_low_entropy['entropy']:.4f}\nVarianz: {normal_low_entropy['variance']:.6f}\nMedian: {normal_low_entropy['median']:.4f}",
    fontsize=12,
    fontweight='bold',
    color='green'
)
axes[1].axis('off')

plt.suptitle(
    'Bildvergleich: Tumor vs. Normal mit Textur-Features',
    fontsize=14,
    fontweight='bold',
    y=0.98
)
plt.tight_layout()
plt.savefig(RESULTS_DIR / '12_image_comparison_with_entropy.png', dpi=150, bbox_inches='tight')
plt.show()

print("PROJEKTANFORDERUNG 3 erfüllt")
print(f"Plot gespeichert: results/figures/12_image_comparison_with_entropy.png")

## 5. Zusätzliche Analyse: Alle Features im Histogramm

Bonus-Visualisierung für besseres Verständnis.

In [ ]:
# Alle 3 Features als Histogramme
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

features = [
    ('entropy', 'Entropie'),
    ('variance', 'Varianz'),
    ('median', 'Median')
]

for idx, (feature, label) in enumerate(features):
    tumor_data = df[df['label'] == 'Tumor'][feature]
    normal_data = df[df['label'] == 'Normal'][feature]
    
    axes[idx].hist(tumor_data, bins=12, alpha=0.6, label='Tumor', color='red', edgecolor='black')
    axes[idx].hist(normal_data, bins=12, alpha=0.6, label='Normal', color='green', edgecolor='black')
    
    axes[idx].axvline(tumor_data.mean(), color='darkred', linestyle='--', linewidth=2)
    axes[idx].axvline(normal_data.mean(), color='darkgreen', linestyle='--', linewidth=2)
    
    axes[idx].set_xlabel(label, fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Häufigkeit', fontsize=11)
    axes[idx].set_title(f'{label}-Verteilung', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Alle Features: Tumor vs. Normal', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / '13_all_features_histograms.png', dpi=150, bbox_inches='tight')
plt.show()

print("Bonus-Plot gespeichert: results/figures/13_all_features_histograms.png")

## 6. Zusammenfassung Teil 3

### Alle Projektanforderungen erfüllt:

1. **Gepaartes Histogramm für Entropie** → `10_entropy_histogram_paired.png`
2. **Korrelations-Heatmap** → `11_correlation_heatmap.png`
3. **Bildvergleich mit Entropie** → `12_image_comparison_with_entropy.png`

### Zusätzliche Visualisierungen:
- Alle Features als Histogramme → `13_all_features_histograms.png`

### Erkenntnisse:

**Aus den Plots ersichtlich:**
- Unterscheiden sich die Entropie-Verteilungen zwischen Tumor und Normal?
- Welche Features korrelieren stark?
- Sind die visuellen Unterschiede in den Bildern erkennbar?

### Nächster Schritt (Teil 4):

- Decision Tree Classifier trainieren
- Feature Importance analysieren
- Modell-Performance evaluieren

---

**Status:** Teil 3 komplett

**Bereit für:** Teil 4 - Machine Learning Klassifikation

---